In [36]:
#!pip install -r ../requirements.txt

In [3]:
import pandas as pd
import numpy as np
import time
import gc
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import evaluate



from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset


import torch


# LOAD DATA

Download the Data from https://www.kaggle.com/datasets/rmisra/news-category-dataset

In [5]:
path = ""
workdata_path = "../working_data/"

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rmisra/news-category-dataset")

print("Path to dataset files:", path)

100%|██████████| 26.5M/26.5M [00:01<00:00, 15.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/rmisra/news-category-dataset/versions/3


In [7]:
# Copy the data to our wor diratory
# !cp  /Users/yhonain/.cache/kagglehub/datasets/rmisra/news-category-dataset/versions/3/News_Category_Dataset_v3.json /Users/yhonain/nlp_task/data/

In [8]:
data = pd.read_json(f'{path}/News_Category_Dataset_v3.json', lines = True)

In [9]:
data.shape

(209527, 6)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  object        
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.6+ MB


In [11]:
pd.DataFrame(data['category'].value_counts()).join(
    pd.DataFrame(data['category'].value_counts(normalize=True))
)

,count,proportion
category,,
POLITICS,35602,0.169916
WELLNESS,17945,0.085645
ENTERTAINMENT,17362,0.082863
TRAVEL,9900,0.047249
STYLE & BEAUTY,9814,0.046839
PARENTING,8791,0.041956
HEALTHY LIVING,6694,0.031948
QUEER VOICES,6347,0.030292
FOOD & DRINK,6340,0.030259


In [12]:
pd.crosstab(data[data['authors'].isin(list(data['authors'].value_counts()[:20].index))]['category'], data[data['authors'].isin(list(data['authors'].value_counts()[:20].index))]['authors'])

authors,,Andy McDonald,Bill Bradley,Carly Ledbetter,Caroline Bologna,Cole Delbyck,Curtis M. Wong,Dana Oliver,David Moye,Dominique Mosbergen,Ed Mazza,Igor Bobic,Julia Brucculieri,Lee Moran,Mary Papenfuss,Michelle Manetti,Nina Golgowski,"Reuters, Reuters",Ron Dicker,Sam Levine
category,,,,,,,,,,,,,,,,,,,,
ARTS,19,0,0,0,0,0,7,0,0,4,0,0,0,0,0,0,0,0,1,0
ARTS & CULTURE,32,4,0,1,1,3,6,0,6,3,6,0,5,41,6,0,7,0,3,0
BLACK VOICES,1008,14,0,1,0,0,4,2,53,6,12,0,0,31,7,0,35,23,40,1
BUSINESS,991,0,0,0,1,0,0,0,2,21,21,1,0,2,25,0,13,294,13,2
COLLEGE,111,0,0,0,0,0,0,0,0,3,2,0,0,0,0,0,2,1,1,0
COMEDY,729,857,167,4,0,9,0,0,59,25,343,0,15,841,129,0,8,0,414,0
CRIME,977,0,0,0,0,0,0,0,97,31,58,1,0,92,76,0,251,45,45,0
CULTURE & ARTS,370,0,0,0,0,0,1,0,0,1,0,0,0,2,0,0,0,1,0,0
DIVORCE,1731,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0


In [13]:

perc = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]
data['news'] = data['headline'] +' -- '+ data['short_description']
pd.concat([
    pd.DataFrame(data['headline'].str.len().describe(percentiles=perc)),
    pd.DataFrame(data['short_description'].str.len().describe(percentiles=perc)),
    pd.DataFrame(data['news'].str.len().describe(percentiles=perc))
], axis=1)

,headline,short_description,news
count,209527.000000,209527.000000,209527.000000
mean,58.415355,114.208670,176.624025
std,18.808506,80.840575,78.552970
min,0.000000,0.000000,4.000000
10%,33.000000,9.000000,79.000000
20%,42.000000,46.000000,112.000000
30%,49.000000,70.000000,137.800000
40%,55.000000,94.000000,158.000000
50%,60.000000,120.000000,174.000000
60%,64.000000,122.000000,187.000000


# DATA PROCESSING

In [14]:
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['category'])

In [15]:
data_work = data[['news', 'label']]


In [16]:
rest, validation = train_test_split(data_work,test_size=0.3 ,random_state=2025, stratify =data['label'])

In [17]:
validation.to_excel(f'{workdata_path}validation.xlsx', index=False)
validation.shape


(62859, 2)

In [18]:
# proportions = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

In [19]:
proportions = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
for p in proportions:
    # Selecciono muestra
    _, v = train_test_split(rest,test_size=p,random_state=2025 ,stratify =rest['label'])
    # Creo columna que diferencia entrenamiento y test
    if p>0.001:
        v_tr, v_ts = train_test_split(v,test_size=0.2,random_state=2025 ,stratify =v['label'])
    else:
        v_tr, v_ts = train_test_split(v,test_size=0.2,random_state=2025 )

    v_tr['part']= 'train'
    v_ts['part']= 'test'
    v = pd.concat([v_tr,v_ts]).sort_index()
    v.to_excel(f'{workdata_path}prop_{p}.xlsx', index=False)
    del(v)
    del(v_ts)
    del(v_tr)
rest_tr, rest_ts = train_test_split(rest,test_size=0.2,random_state=2025 ,stratify =rest['label'])
rest_tr['part']= 'train'
rest_ts['part']= 'test'
rest = pd.concat([rest_tr,rest_ts]).sort_index()
rest.to_excel(f'{workdata_path}prop_1.xlsx', index=False)

# BERT PIPELINE
Executed in Runpod with a RTX 4090

### AUXILIAR FUNCTIONS

In [33]:
torch.cuda.is_available()

True

In [26]:
def tokenize_function(df,tokenizer,padding,truncation):
    """
    Function to tokenize data and give it the structure needed
    """
    tokens = tokenizer(df["news"], padding=padding, truncation=truncation)
    tokens["labels"] = df["label"]
    return tokens

def performance(trainer,p,tokenizer):
    """
    Function to compute the performace of the model on validation data
    """
    padding="max_length"
    truncation=True
    metric = evaluate.load("f1")

    # Tokenize validation data
    val_dataset = Dataset.from_pandas(validation)
    val_tok = val_dataset.map(lambda batch: tokenize_function(batch, tokenizer, padding, truncation), batched=True)

    # Predictions
    predictions = trainer.predict(val_tok)
    pred_labels = np.argmax(predictions.predictions, axis=1)

    # Performance
    report = metric.compute(references = predictions.label_ids, predictions = pred_labels, average=None)

    df_report = pd.DataFrame({
        "Category": label_encoder.classes_,
        "F1-Score": report["f1"]
    })    
    # report = classification_report(predictions.label_ids, pred_labels, target_names=label_encoder.classes_, output_dict=True)
    # df_report = pd.DataFrame(report).transpose()

    # Guardarlo en un archivo Excel o CSV
    # df_report.to_csv(f"../performance/bert_{p}.csv")
    df_report.to_csv(f"../performance/bert_{p}.csv")

### TRAINING FUNCTION

In [28]:
def fine_tune(p):
    """
    Function to fine tune BERT
    """
    mode_checkpoint="bert-base-cased"
    padding="max_length"
    truncation=True
    save_checkpoints = "../checkpoints"
    data_tr = pd.read_excel(f'{workdata_path}prop_{p}.xlsx')
    num_labels = len(data_tr["label"].unique())

    # Define the tokenizer and the model
    tokenizer = AutoTokenizer.from_pretrained(mode_checkpoint)
    model = AutoModelForSequenceClassification.from_pretrained(mode_checkpoint, num_labels=num_labels)

    # # Force the model top use GPU
    # device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    # model.to(device)

    # Train and validation set
    train_df = data_tr[data_tr['part']=='train'][['news','label']]
    test_df = data_tr[data_tr['part']=='test'][['news','label']]


    # Tokenize the data
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)
    train_tok = train_dataset.map(lambda batch: tokenize_function(batch, tokenizer, padding, truncation), batched=True)
    test_tok = test_dataset.map(lambda batch: tokenize_function(batch, tokenizer, padding, truncation), batched=True)

    # Set the training arguments
    training_args = TrainingArguments(
        output_dir=save_checkpoints,
        eval_strategy="epoch",
        per_device_train_batch_size=8,
        fp16=True if torch.cuda.is_available() else False,
        logging_steps=10,
        save_strategy="no",
        report_to="none",
    )

    # Instanciate the training object
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tok,
        eval_dataset=test_tok
    )

    # Train the model
    trainer.train()
    return trainer, tokenizer

In [29]:
proportions.append(1)

In [31]:
proportions = proportions[:-1]
proportions


[0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

### BERT LOOP

In [34]:
proportions

[0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

In [32]:



for prop in proportions:
    t_inicial = time.time()
    print(f"Proceso proporción: {prop}")
    trained, tokenizer = fine_tune(prop)
    print(f"    Finaliza entrenamiento en {round((time.time()-t_inicial)/60)} minutos")
    performance(trained,prop,tokenizer)
    print(f"    Finaliza evaluación en {round((time.time()-t_inicial)/60)} minutos")

    del(trained)
    gc.collect()
    torch.cuda.empty_cache()


Proceso proporción: 0.001


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/117 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,3.591800,3.636848
2,3.351200,3.588998
3,3.097000,3.566536


...

    Finaliza entrenamiento en 29 minutos


Map:   0%|          | 0/62859 [00:00<?, ? examples/s]

    Finaliza evaluación en 32 minutos
Proceso proporción: 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyError: 'part'

In [39]:

for prop in proportions[-1:]:
    t_inicial = time.time()
    print(f"Proceso proporción: {prop}")
    trained, tokenizer = fine_tune(prop)
    print(f"    Finaliza entrenamiento en {round((time.time()-t_inicial)/60)} minutos")
    performance(trained,prop,tokenizer)
    print(f"    Finaliza evaluación en {round((time.time()-t_inicial)/60)} minutos")

    del(trained)
    gc.collect()
    torch.cuda.empty_cache()

Proceso proporción: 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/117334 [00:00<?, ? examples/s]

Map:   0%|          | 0/29334 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,1.063600,1.153072
2,0.848400,1.072012
3,0.390500,1.191224


    Finaliza entrenamiento en 59 minutos


Map:   0%|          | 0/62859 [00:00<?, ? examples/s]

    Finaliza evaluación en 62 minutos


# RAG

We are not splitting the documents because they are shorts (<1500 chars)